In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install ftfy

Bibliotecas:

In [ ]:
#Para o uso geral
import random
import numpy as np
import pandas as pd
import copy 
import time
from scipy.stats import uniform
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import io
import sklearn.metrics as metr
#Para o processamento de textos
from ftfy import fix_text
import string
import re
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

#Para Machine Learning e NLP
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# **Doc2Vec:**

Importando dados brutos:

In [ ]:
data_bruto = pd.read_csv("../input/sentiment-analysis-pmr3508/data_train.csv", sep=',')

Verificando como são esses dados:

In [ ]:
data_bruto.head()

Os dados contém duas colunas, uma com um texto que é a análise do IMDB e outra indicando se o texto contém emoções positivas (1) ou negativas (0)

Verificando se há duplicatas: (sim, há)

In [ ]:
data_bruto.loc[data_bruto.duplicated(subset='review', keep=False)==True].sort_values(by='review').head(10)

Removendo então os dados duplicados:

In [ ]:
data_bruto=data_bruto.drop_duplicates(subset='review', keep='first')

Verificando a distribuição das criticas:

In [ ]:
data_bruto.loc[:,'positive'].value_counts()

É possível ver que os dados coletados para treino estão distribuidos quase igualmente entre criticas positivas e negativas.

Separando os textos e a variável booleana indicadora da critica:

In [ ]:
X = data_bruto.loc[:,'review'].tolist()
y = np.array(data_bruto.loc[:,'positive'].tolist())

Limpando e padronizando os textos:

In [ ]:
def clean(text):
    txt=text.replace("<br />"," ") #retirando tags
    txt=fix_text(txt) #consertando Mojibakes (Ver https://pypi.org/project/ftfy/)
    txt=txt.lower() #passando tudo para minúsculo
    txt=txt.translate(str.maketrans('', '', string.punctuation)) #retirando toda pontuação
    txt=txt.replace(" — ", " ") #retirando hífens
    txt=re.sub("\d+", ' <number> ', txt) #colocando um token especial para os números
    txt=re.sub(' +', ' ', txt) #deletando espaços extras
    return txt
X = [clean(x) for x in X]

Separando o texto em palavras:

In [ ]:
X = [x.split() for x in X]

Finalmente, transformando os textos em vetores através do Doc2Vec

In [ ]:
def emb(txt, model, normalize=False): 
    model.random.seed(42)
    x=model.infer_vector(txt, steps=20)
    
    if normalize: return(x/np.sqrt(x@x))
    else: return(x)

In [ ]:
d2v = Doc2Vec.load("../input/sentiment-analysis-pmr3508/doc2vec")
X = [emb(x, d2v) for x in X]  
X = np.array(X)                  

Estrutura dos dados finais para treino:

In [ ]:
X.shape, y.shape

# **Treinando redes neurais da Scikit-Learn:**

1 camada escondida:

Usando o RandomizedSearch para procurar um bom classificador com os seguintes hyperparametros ajustáveis: hidden_layer_sizes,learning_rate, activation e alpha(L2 penalty) 

In [ ]:
n_neuronios_simples = tuple((i,) for i in range(1, 200))
mlp = MLPClassifier(early_stopping = True)
hyperparameters = dict(hidden_layer_sizes = n_neuronios_simples, learning_rate = ['constant','invscaling', 'adaptive'], activation=['identity','relu','logistic'],alpha = (0.000001,0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000))
achado = RandomizedSearchCV(mlp, hyperparameters, scoring='roc_auc', n_iter=20, n_jobs=-1, random_state=0, verbose=2)
camada_simples = achado.fit(X, y)

Os melhores parametros foram:

In [ ]:
camada_simples.best_params_

Melhor área da curva ROC foi:

In [ ]:
camada_simples.best_score_

Treinando então o classificador final de uma camada escondida

In [ ]:
camada_simples_melhor = MLPClassifier(activation=camada_simples.best_params_['activation'], early_stopping = True, hidden_layer_sizes=camada_simples.best_params_['hidden_layer_sizes'], learning_rate=camada_simples.best_params_['learning_rate'],alpha=camada_simples.best_params_['alpha'],random_state=90)
camada_simples_melhor.fit(X, y)


2 camada escondida:

Usando o RandomizedSearch para procurar um bom classificador com os seguintes hyperparametros ajustáveis: hidden_layer_sizes,learning_rate, activation e alpha(L2 penalty) 

In [ ]:
n_neuronios_dupla = []
for i in range(1, 200):
    for j in range(1, 200):
        n_neuronios_dupla.append((i,j,))
n_neuronios_dupla = tuple(n_neuronios_dupla)
mlp = MLPClassifier(early_stopping = True)
hyperparameters = dict(hidden_layer_sizes = n_neuronios_dupla, learning_rate = ['constant','invscaling', 'adaptive'], activation=['identity','relu','logistic'],alpha = (0.000001,0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000))
achado = RandomizedSearchCV(mlp, hyperparameters, scoring='roc_auc', n_iter=20, n_jobs=-1, random_state=0, verbose=2)
camada_dupla = achado.fit(X, y)

Os melhores parametros foram:

In [ ]:
camada_dupla.best_params_

Melhor área da curva ROC foi:

In [ ]:
camada_dupla.best_score_

Treinando então o classificador final de uma camada escondida

In [ ]:
camada_dupla_melhor = MLPClassifier(activation=camada_dupla.best_params_['activation'], early_stopping = True, hidden_layer_sizes=camada_dupla.best_params_['hidden_layer_sizes'], learning_rate=camada_dupla.best_params_['learning_rate'],alpha=camada_dupla.best_params_['alpha'],random_state=90)
camada_dupla_melhor.fit(X, y)


# **Treinando um classificador usando Regressão Logística**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
logreg = LogisticRegression(solver='liblinear',random_state=90)
hyperparams = dict(C=np.linspace(0,10,100), penalty=['l2', 'l1'])
                     
classificador_lr = RandomizedSearchCV(logreg, hyperparams, scoring='roc_auc', n_iter=50, cv=2, n_jobs=-1, random_state=0, verbose=2)
busca_logreg = classificador_lr.fit(X, y)

Os melhores parametros foram:

In [ ]:
busca_logreg.best_params_

Melhor área da curva ROC foi:

In [ ]:
busca_logreg.best_score_

Treinando então o classificador final de uma camada escondida

In [ ]:
classificador_lr = LogisticRegression(solver='liblinear',random_state=90,C=busca_logreg.best_params_['C'],penalty=busca_logreg.best_params_['penalty'])
classificador_lr.fit(X, y)


# **Treinando um classificador usando LDA**

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [ ]:
lda = LinearDiscriminantAnalysis()
hyperparams = dict(store_covariance=['True','False'] ,solver=['svd', 'lsqr', 'eigen'])
                     
classificador_lda = RandomizedSearchCV(lda, hyperparams, scoring='roc_auc', n_iter=50, cv=2, n_jobs=-1, random_state=0, verbose=2)
busca_lda = classificador_lda.fit(X, y)

Os melhores parametros foram:

In [ ]:
busca_lda.best_params_

Melhor área da curva ROC foi:

In [ ]:
busca_lda.best_score_

Treinando então o classificador final de uma camada escondida

In [ ]:
classificador_lda = LinearDiscriminantAnalysis(store_covariance=busca_lda.best_params_['store_covariance'] ,solver=busca_lda.best_params_['solver'])
classificador_lda.fit(X, y)

# **Treinando um classificador usando QDA**

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


In [ ]:
qda = QuadraticDiscriminantAnalysis()
hyperparams = dict(store_covariance=['True','False'] ,reg_param=[0,0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1])
                     
classificador_qda = RandomizedSearchCV(qda, hyperparams, scoring='roc_auc', n_iter=50, cv=2, n_jobs=-1, random_state=0, verbose=2)
busca_qda = classificador_qda.fit(X, y)

Os melhores parametros foram:

In [ ]:
busca_qda.best_params_

Melhor área da curva ROC foi:

In [ ]:
busca_qda.best_score_

Treinando então o classificador final de uma camada escondida

In [ ]:
classificador_qda = QuadraticDiscriminantAnalysis(store_covariance=busca_qda.best_params_['store_covariance'] ,reg_param=busca_qda.best_params_['reg_param'])
classificador_qda.fit(X, y)

# **Escolhendo melhor modelo**

In [ ]:
data_bruto_teste = pd.read_csv("../input/sentiment-analysis-pmr3508/data_test1.csv", sep=',')
X_teste1 = data_bruto_teste.loc[:,'review'].tolist()
y_teste1 = np.array(data_bruto_teste.loc[:,'positive'].tolist())
X_teste1 = [clean(x) for x in X_teste1]
X_teste1 = [x.split() for x in X_teste1]
X_teste1 = [emb(x, d2v) for x in X_teste1]  
X_teste1 = np.array(X_teste1) 

data_bruto_teste = pd.read_csv("../input/sentiment-analysis-pmr3508/data_test2_X.csv", sep=',')
X_teste2 = data_bruto_teste.loc[:,'review'].tolist()
X_teste2 = [clean(x) for x in X_teste2]
X_teste2 = [x.split() for x in X_teste2]
X_teste2 = [emb(x, d2v) for x in X_teste2]  
X_teste2 = np.array(X_teste2) 

In [ ]:
X_teste1.shape, y_teste1.shape

In [ ]:
X_teste2.shape

In [ ]:
auc_1c=roc_auc_score(y_teste1, camada_simples_melhor.predict_proba(X_teste1)[:,1])
auc_2c=roc_auc_score(y_teste1, camada_dupla_melhor.predict_proba(X_teste1)[:,1])
auc_lr=roc_auc_score(y_teste1, classificador_lr.predict_proba(X_teste1)[:,1])
auc_lda=roc_auc_score(y_teste1, classificador_lda.predict_proba(X_teste1)[:,1])
auc_qda=roc_auc_score(y_teste1, classificador_qda.predict_proba(X_teste1)[:,1])

In [ ]:
print('AUC:MLP 1Camada:{:.4f}'.format(auc_1c))
print('AUC:MLP 2Camada:{:.4f}'.format(auc_2c))
print('AUC:Linear Regression:{:.4f}'.format(auc_lr))
print('AUC:LDA:{:.4f}'.format(auc_lda))
print('AUC:QDA:{:.4f}'.format(auc_qda))

O MLP de duas camadas foi o classificador com maior AUC**

In [ ]:
submissão = {'positive': camada_dupla_melhor.predict_proba(X_teste2)[:,1]}
submissão = pd.DataFrame(submissão)
submissão.to_csv("PMR3508-2020-90_out.csv", index_label="Id")